In [1]:
import torch
import spacy
from selfcheckgpt.modeling_mqag import SelfCheckMQAG
from selfcheckgpt.modeling_bertscore import SelfCheckBERTScore

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
passage = """
Michael Alan Weiner (born March 31, 1942), better known by his professional name Michael Savage, is an American radio host, author, activist, nutritionist, and conservative political commentator. He is the host of The Savage Nation, a nationally syndicated talk show that aired on Talk Radio Network across the United States until 2012, and in 2009 was the second most listened-to radio talk show in the country with an audience of over 20 million listeners on 400 stations across the United States. Since October 23, 2012, Michael Savage has been syndicated by Cumulus Media Networks. He holds master's degrees from the University of Hawaii in medical botany and medical anthropology, and a Ph.D. from the University of California, Berkeley in nutritional ethnomedicine. As Michael Weiner, he has written books on nutrition, herbal medicine, and homeopathy.
""".replace("\n", " ").strip()

In [4]:
print(passage)

Michael Alan Weiner (born March 31, 1942), better known by his professional name Michael Savage, is an American radio host, author, activist, nutritionist, and conservative political commentator. He is the host of The Savage Nation, a nationally syndicated talk show that aired on Talk Radio Network across the United States until 2012, and in 2009 was the second most listened-to radio talk show in the country with an audience of over 20 million listeners on 400 stations across the United States. Since October 23, 2012, Michael Savage has been syndicated by Cumulus Media Networks. He holds master's degrees from the University of Hawaii in medical botany and medical anthropology, and a Ph.D. from the University of California, Berkeley in nutritional ethnomedicine. As Michael Weiner, he has written books on nutrition, herbal medicine, and homeopathy.


In [12]:
passage_edited = """
Michael Alan Weiner (born January 31, 1952), better known by his professional name Michael King, is an American radio host, author, activist, nutritionist, and conservative political commentator. He is the host of The Savage Nation, a nationally syndicated talk show that aired on Talk Radio Network across the United States until 2012, and in 2009 was the second most listened-to radio talk show in the country with an audience of over 20 million listeners on 400 stations across the United States. Since October 23, 2002, Michael King has been syndicated by Virgin Media Networks. He holds master's degrees from the University of Hawaii in medical botany and medical anthropology, and a Ph.D. from the University of California, San Diego in nutritional ethnomedicine. As Michael Weiner, he has written books on nutrition, herbal medicine, and homeopathy.
""".replace("\n", " ").strip()

In [13]:
print(passage_edited) # edit sent1, sent3, sent4 to inaccurate information

Michael Alan Weiner (born January 31, 1952), better known by his professional name Michael King, is an American radio host, author, activist, nutritionist, and conservative political commentator. He is the host of The Savage Nation, a nationally syndicated talk show that aired on Talk Radio Network across the United States until 2012, and in 2009 was the second most listened-to radio talk show in the country with an audience of over 20 million listeners on 400 stations across the United States. Since October 23, 2002, Michael King has been syndicated by Virgin Media Networks. He holds master's degrees from the University of Hawaii in medical botany and medical anthropology, and a Ph.D. from the University of California, San Diego in nutritional ethnomedicine. As Michael Weiner, he has written books on nutrition, herbal medicine, and homeopathy.


In [5]:
sample1 = """
Michael Alan Weiner (born March 31, 1942), better known by his professional name Michael Savage, is an American radio host, author, and political commentator. He is the host of The Savage Nation, a nationally syndicated talk show broadcast on Talk Radio Network across the United States, and on Comet in Canada. He holds master's degrees from the University of Hawaii in medical botany and medical anthropology, and a Ph.D. from the University of California, Berkeley in epidemiology and nutrition sciences. Savage has summarized his political philosophy in three words: borders, language, and culture. He has characterized his views as conservative nationalism, while critics have characterized them as fascism. He is a fierce critic of liberalism, and the War on Drugs. He believes that political correctness has "ruined the United States" and that the American government has spent too much money on social programs throughout its history.
""".replace("\n", " ").strip()

sample2 = """
Michael A. Savage (born Michael Alan Weiner; March 31, 1942) is an American radio host, author, activist, nutritionist, and conservative political commentator. He is the host of The Savage Nation, a nationally syndicated talk show that aired on Talk Radio Network across the United States until 2012. The Savage Nation has a combined audience of over 20 million listeners on 400 stations across the United States and is one of the top-rated shows in the country. He is also the host of The Michael Savage Show, a daily podcast and webcast heard around the world. He has been called "the godfather of Trumpmania".
Savage has written more than 25 books and is an outspoken supporter of traditional American values and break-down of the culture of political correctness. He advocates an "America first" foreign policy and rigorous opposition to illegal immigration. For his social and political views, Savage has been described as a conservative and a right-wing populist. He has campaigned for numerous political causes, including opposition to same-sex marriage and the legalization of marijuana.
""".replace("\n", " ").strip()

sample3 = """
Michael Alan Weiner (born March 31, 1942),[2] better known by his professional name Michael Savage, is an American radio host, author, activist, nutritionist, and conservative political commentator. He is the host of The Savage Nation, a nationally syndicated talk show that aired on Talk Radio Network across the United States until 2012, and in 2009 was the second most listened-to radio talk show in the country with an audience of over 20 million listeners on 400 stations across the United States. Since October 23, 2012, Michael Savage has been syndicated by Cumulus Media Networks. He holds master's degrees from the University of Hawaii in medical botany and medical anthropology, and a Ph.D. from the University of California, Berkeley in nutritional ethnomedicine. As Michael Weiner, he has written books on nutrition, herbal medicine, and homeopathy.
Savage has summarized his political philosophy in three words: borders, language, and culture. He believes that the United States should end foreign aid, impose much stricter immigration laws, deport illegal immigrants, and put an end to birthright citizenship. Savage has characterized his views as conservative nationalism, free
""".replace("\n", " ").strip()

## Initialization
- Calling `SelfCheckMQAG()` first time will take some time as it will download the generation and answering models from HuggingFace Model Hub.

In [6]:
selfcheck_mqag = SelfCheckMQAG(device)
selfcheck_bertscore = SelfCheckBERTScore()

SelfCheck-MQAG initialized to device cuda
SelfCheck-BERTScore initialized


## Running Inference

### 1. Uneditted/Original Passage

In [7]:
nlp = spacy.load("en_core_web_sm")
sentences = [sent for sent in nlp(passage).sents] # List[spacy.tokens.span.Span]
sentences = [sent.text.strip() for sent in sentences if len(sent) > 3]

In [8]:
print("SelfCheck running on {} sentences...".format(len(sentences)))

SelfCheck running on 5 sentences...


In [9]:
sent_scores_mqag = selfcheck_mqag.predict(
    sentences,
    passage,
    [sample1, sample2, sample3],
    num_questions_per_sent = 5,
    scoring_method = 'bayes_with_alpha',
    beta1 = 0.8, beta2 = 0.8,
)

In [10]:
sent_scores_bertscore = selfcheck_bertscore.predict(
    sentences,
    [sample1, sample2, sample3],
)

In [14]:
# Uneditted passge
print("MQAG\tBERTScore")
for s1, s2 in zip(sent_scores_mqag, sent_scores_bertscore):
    print("{:.4f}\t{:.4f}".format(s1, s2))

MQAG	BERTScore
0.0230	0.0236
0.3281	0.0384
0.3192	0.0889
0.3882	0.0589
0.4182	0.0798


### 2. Editted Passage

In [15]:
sentences = [sent for sent in nlp(passage_edited).sents] # List[spacy.tokens.span.Span]
sentences = [sent.text.strip() for sent in sentences if len(sent) > 3]

In [16]:
print("SelfCheck running on {} sentences...".format(len(sentences)))

SelfCheck running on 5 sentences...


In [17]:
sent_scores_mqag = selfcheck_mqag.predict(
    sentences,
    passage,
    [sample1, sample2, sample3],
    num_questions_per_sent = 5,
    scoring_method = 'bayes_with_alpha',
    beta1 = 0.8, beta2 = 0.8,
)

In [18]:
sent_scores_bertscore = selfcheck_bertscore.predict(
    sentences,
    [sample1, sample2, sample3],
)

In [19]:
# Editted passge [seent1, sent3, sent4 were editted]
print("MQAG\tBERTScore")
for s1, s2 in zip(sent_scores_mqag, sent_scores_bertscore):
    print("{:.4f}\t{:.4f}".format(s1, s2))

MQAG	BERTScore
0.4071	0.0329
0.1585	0.0384
0.4999	0.1050
0.4284	0.0632
0.2793	0.0798
